In [11]:
import pickle
import numpy as np
import bcolz
import openke
# from openke.config import Trainer, Tester
# from openke.module.model import TransE
# from openke.module.loss import MarginLoss
# from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader
import json
import collections
from scipy.sparse import csc_matrix
import scipy

In [12]:
train_dataloader = TrainDataLoader(
	in_path = "./benchmarks/FB60K/",
	nbatches = 100,
	threads = 8,
	sampling_mode = "normal",
	bern_flag = 1,
	filter_flag = 1,
	neg_ent = 25,
	neg_rel = 0)
entity_size = train_dataloader.get_ent_tot()

In [13]:
entity_size

69515

In [14]:
base_path = '/home/ubuntu/text-pwrd-kg-reasoning/OpenKE/benchmarks/FB60K/'
entity2id = {}
with open("/home/ubuntu/text-pwrd-kg-reasoning/OpenKE/benchmarks/FB60K/entity2id.txt") as f:
    line = f.readline().split()
    max_id = int(line[0])
    line = f.readline().split()
    while line:
        entity2id[line[0]] = int(line[1])
        line = f.readline().split()

In [15]:
with open(base_path + 'train.json') as f:
    train_file = json.load(f) 

In [16]:
similarity_matrix = collections.Counter()
for data in train_file:
#     head_id = "/" + "/".join(data['head']['id'].split("."))
    head_entity =  data['head']['id']
    tail_entity = data['tail']['id']
    head_idx = entity2id[head_entity]
    tail_idx = entity2id[tail_entity]
    fin_key = str(head_idx) + '_' + str(tail_idx)                                                                                      
    similarity_matrix[fin_key] += 1
similarity_matrix = dict(similarity_matrix)

In [41]:
max_val = np.log(max(list(similarity_matrix.values())))
min_val = np.log(min(list(similarity_matrix.values())))

In [59]:
row = []
col = []
data = []
for k,v in similarity_matrix.items():
    k = k.split("_")
    i = int(k[0])
    j = int(k[1])
    row.append(i)
    col.append(j)
    v = np.log(v)
    v = (v - min_val) / (max_val - min_val)
    data.append(v)

In [60]:
sim_matrix = csc_matrix((data, (row, col)), shape=(entity_size, entity_size))

In [61]:
base_path = '/home/ubuntu/text-pwrd-kg-reasoning/OpenKE/benchmarks/FB60K/'
scipy.sparse.save_npz(base_path + 'sparse_matrix.npz', sim_matrix)
sparse_matrix = scipy.sparse.load_npz(base_path + 'sparse_matrix.npz')

In [62]:
rows, cols = sparse_matrix.nonzero()
head = np.load("head.npy")
tail = np.load("tail.npy")

In [63]:
c = np.array(sparse_matrix[head, tail]).reshape(-1)

In [64]:
len(rows)

55658

In [58]:
len(rows)

291699

In [65]:
inds = np.argwhere(c!=0)
c = c[inds]

In [66]:
len(c)

171

In [67]:
for i,val in enumerate(c):
    if val != sparse_matrix[head[inds[i]], tail[inds[i]]]:
        print(i)

In [68]:
inds[0]

array([15])

In [71]:
print(head[inds[0]])
print(tail[inds[0]])
print(sparse_matrix[head[inds[0]],tail[inds[0]]])
c[0]

[8753]
[98]
[[0.08137447]]


array([0.08137447])